### Thsi notebook shows how to write simple Prompt templates custom Agent tools using Langchain

### 1. Without Prompt template

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_ollama import OllamaLLM


# Initialize the Ollama model (make sure it's running locally)
llm = OllamaLLM(model ="llama2") 

# Wrap into a Runnable using a simple lambda or chain
chain = RunnableLambda(lambda prompt: llm.invoke(prompt))

# Invoke the model
response = chain.invoke("What's the capital of Canada?")
print(response)

### 2. 'With' Prompt template and passable parameters.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Create a prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

# Chain it
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
result = chain.invoke("Canada")
print(result)

In [ ]:
# Run the chain
result = chain.invoke("India")
print(result)

In [ ]:
result = chain.invoke("US")
print(result)

###  Define Agent Tool with Parameters

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.llms import Ollama

# Define the tool (manually)
def get_squared_func(number: str) -> str:
    """Takes a number as a string and returns its square. Handles messy input."""
    # Extract the first integer from the string
    import re
    match = re.search(r'\d+', number)
    if not match:
        return "Invalid input: No number found"
    num = int(match.group())
    return str(num ** 2)


get_squared_tool = Tool(
    name="get_squared",
    func=get_squared_func,
    description="Takes an number string as input and returns its square."
)

llm = Ollama(model="llama2")



### Set up the agent 

In [ ]:
agent = initialize_agent(
    tools=[get_squared_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2, 
     early_stopping_method="generate"  # Optional: finish with best guess
)


### Test the tool

In [ ]:
## # Now this prompt style is valid
# agent.run("What is the square of 12?")
# directly calling the tool

get_squared_tool.run('15')

'225'

### Building Agent tool as a Runnable pipeline

In [88]:
import re
from langchain_core.runnables import RunnableLambda, RunnableSequence
from langchain_community.llms import Ollama

# Step 1: Define the get_squared tool
def get_squared(number: int) -> int:
    return number * number

# Step 2: Parse numbers from input
def extract_numbers(text: str) -> list[int]:
    return [int(n) for n in re.findall(r'\d+', text)]

# Step 3: Build a LangChain Runnable pipeline
# - Input: user prompt
# - Output: formatted result string
pipeline = (
    RunnableLambda(lambda text: extract_numbers(text))  # ["12", "36"]
    | RunnableLambda(lambda nums: [(n, get_squared(n)) for n in nums])  # [(12, 144), (36, 1296)]
    | RunnableLambda(lambda results: ", ".join(f"{n}² = {sq}" for n, sq in results))  # "12² = 144, 36² = 1296"
)

# Optional: use Ollama to add LLM reasoning before/after
llm = Ollama(model="llama2")

# Run the pipeline
user_input = "What is the square of 12 and 36?"
result = pipeline.invoke(user_input)
print(result)


12² = 144, 36² = 1296


In [89]:
# Run the pipeline
user_input = "What is the square of 15?"
result = pipeline.invoke(user_input)
print(result)

15² = 225


### Explain the output using Prompting

In [90]:
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap

# Add an LLM explanation step
explanation_prompt = PromptTemplate.from_template(
    "Given these squared results: {results}, explain them in a friendly sentence."
)

chain = (
    pipeline
    | RunnableMap({"results": lambda x: x})  # name the result for the prompt
    | explanation_prompt
    | llm
)

print(chain.invoke(user_input))



 Hey there! So, you know how we multiplied 15 by itself? Well, when you multiply a number by itself, the result is always going to be bigger than the original number! Isn't that cool? So, in this case, 15 multiplied by itself equals 225. That means that 15 squared is equal to 225! Pretty neat, right?
